In [115]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot
from math import floor
import matplotlib.pyplot as plt

In [116]:
NUM_INPUTS = 15
NUM_OUTPUTS = 11

# Load the dataset input and output
inputNums = np.loadtxt('../dataset/0_input.csv', delimiter=',') # (num training points, 13)
y = np.loadtxt('../dataset/0_output.csv', delimiter=',') # (num training points, 11)
X = np.zeros((len(inputNums), 480, 640, 1)) # (num training points, 480, 640)

# For each of the training examples there should be exactly 1 image
for i in range(0, len(inputNums)):
    image = Image.open('../dataset/img_' + str(i) + '.jpg') 
    imageArr = np.asarray(image).reshape(480, 640, 1) # Converts the image into a 2D 480*640 array
    X[i] = imageArr / 255


In [121]:
numerical_inputs = Input(shape=(NUM_INPUTS,))
image_inputs = Input(shape=(480, 640, 1))

# Process the image data first
x_image = Conv2D(32, (7, 7), strides=(4, 4), padding='same', activation='relu')(image_inputs)
x_image = Conv2D(64, (5, 5), strides=(4, 4), padding="same", activation="relu")(x_image)
x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Conv2D(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Conv2D(256, (3, 3), strides=(2, 2), padding="same", activation="relu")(x_image)
x_image = Model(image_inputs, x_image)
x_image = Flatten()(x_image.output)

print(x_image.shape)
print(numerical_inputs.shape)

# Combine the latent space representation with the numerical input
combined = concatenate([x_image, numerical_inputs])
print(combined.shape)
x_combined = Dense(300, activation='relu')(combined)
x_combined = Dense(60, activation='relu')(x_combined)
outputs = Dense(11, activation='linear')(x_combined)

model = Model([image_inputs, numerical_inputs], outputs)

model.compile(loss='mse', optimizer='adam', metrics=['mae'])
history = model.fit([X, inputNums], y, epochs=100, batch_size=32, validation_split=0.2)

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

(None, 5120)
(None, 15)
(None, 5135)
Train on 593 samples, validate on 149 samples
Epoch 1/100
593/593 [==============================] - 6s 10ms/sample - loss: 6.0204 - mae: 1.2777 - val_loss: 8.0235 - val_mae: 1.6634
Epoch 2/100
593/593 [==============================] - 4s 7ms/sample - loss: 2.7009 - mae: 0.9194 - val_loss: 5.4703 - val_mae: 1.4265
Epoch 3/100
593/593 [==============================] - 4s 7ms/sample - loss: 0.7925 - mae: 0.6269 - val_loss: 3.4888 - val_mae: 1.1323
Epoch 4/100
593/593 [==============================] - 4s 7ms/sample - loss: 0.5267 - mae: 0.5011 - val_loss: 3.0328 - val_mae: 1.1777
Epoch 5/100
593/593 [==============================] - 4s 7ms/sample - loss: 0.4304 - mae: 0.4465 - val_loss: 2.3012 - val_mae: 1.0269
Epoch 6/100
593/593 [==============================] - 4s 7ms/sample - loss: 0.3503 - mae: 0.3986 - val_loss: 2.0941 - val_mae: 0.9892
Epoch 7/100
593/593 [==============================] - 5s 8ms/sample - loss: 0.2667 - mae: 0.3468 - val_lo

KeyboardInterrupt: 